In [17]:
import simpy
import pandas as pd
import numpy as np
import random

## `entities.py` ##

Took attributes that should be for BuiltCapital() / Residence()
Added self.money_to_rebuid, self.income
Changed self.insurance_coverage definition to point to household['Insurance']
Added self.residence
Delete io stuff and moved to io.py

In [18]:
# import pandas as pd
# import numpy as np

class Household(object):
    """Define a Household class with attributes and visualization methods.

        Methods have yet to be defined, but will likely include putting data into
        a pandas dataframe and having the ability to show recovery trajectories
        for each household
    """
    def __init__(self, simulation, household):

        # Household Attributes
        self.household = household
        self.name = household['Name']  # Name assigned to household
        self.income = household['Income'] # --+ added +--
        self.savings = household['Savings'] # --% Modified to read input data %--
        self.insurance = household['Insurance'] # --% Modified to read input data %--
        
        
        #Outputs
        self.story = [] # The story of events for each household
        self.inspection_put = 0.0 # Time put request in for house inspection
        self.inspection_get = 0.0 # Time get  house inspection
        self.claim_put = 0.0 # Time put request in for insurance settlement
        self.claim_get = 0.0 # Time get insurance claim settled
        self.claim_payout = 0.0 # Amount of insurance claim payout
        self.assistance_put = 0.0 # Time put request in for FEMA assistance
        self.assistance_get = 0.0 # Time get FEMA assistance
        self.assistance_request = 0.0 # Amount of money requested from FEMA
        self.assistance_payout = 0.0 # Amount of assistance provided by FEMA
        self.money_to_rebuild = self.savings # Total funds available to household to rebuild house
        self.house_put = 0.0 # Time put request in for house rebuild
        self.house_get = 0.0 # Time get house rebuild completed
        self.loan_put = 0.0 # Time put request for loan
        self.loan_get = 0.0 # Time get requested loan
        self.loan_amount = 0.0 # Amount of loan
        self.permit_put = 0.0 # Time put request for building permit
        self.permit_get = 0.0 # Time get requested building permit
        
        # Function calls
        self.residence = Residence(simulation, household) # Assign residence to the household
        self.setStory() # Start stories with non-disaster attributes
    
    def setStory(self):
        
        # Start stories with non-disaster attributes
        self.story.append(
        '{0} lives in a {1} bedroom {2} Home ({3}). '.format(self.name, 
                                                        self.residence.bedrooms, 
                                                        self.residence.occupancy,
                                                        self.residence.address
                                                        )
        )

    def story_to_text(self): # --% modified %--
        '''
            Function to join the strings within the story list created during DESaster processes
        '''
        
        return ''.join(self.story)


## `capitals.py` ##

Call to Containers() was missing arguments. If only two is given with no keywords, second argument is assumed to be the capacity, not the initial level. To have capacity default to infinit and set an initial level, need to use "init" keyword. Changed to:

`simpy.Container(simulation, init=financial_cap_data['fema aid'])`

In [19]:
# import simpy
# import csv

class HumanCapital(): # --% created a separate class for just human capitals %--
    """This is the main resource data structure.
    
        Provide a data dictionary. The dictionary MUST include all the
        resources listed below. 
    """
    def __init__(self, simulation, human_cap_data): # --% changed argument name %--
                
        self.data = human_cap_data
        
        ## HUMAN CAPITALS ## --% changed to make reference to human_cap_data %--
        
        try:
            self.inspectors = simpy.Resource(simulation, human_cap_data['inspectors'])
            self.insurance_adjusters = simpy.Resource(simulation, human_cap_data['insurance adjusters'])
            self.fema_processors = simpy.Resource(simulation, human_cap_data['fema processors'])
            self.permit_processors = simpy.Resource(simulation, human_cap_data['permit processors'])
            self.contractors = simpy.Resource(simulation, human_cap_data['contractors'])
            self.loan_processors = simpy.Resource(simulation, human_cap_data['loan processors'])
            self.engineers = simpy.Resource(simulation, human_cap_data['engineers'])
        except ValueError as e:
            
            print ("You are missing a config value, or your value is zero. All" 
                    "values must have a positive number, see error: {0}".format(e))
        except KeyError as f:
            print ("You are missing {0} as a capital. Please re-add in your"
                    " config file".format(f))
                    
        except Exception as g:
            print ("Something went really wrong, capitals failed to load."
                    " See error {0}".format(g))

class FinancialCapital(): # --% created a separate class for just financial capitals %--
        def __init__(self, simulation, financial_cap_data): # --% changed/added arguments %--
            
            self.fema_aid = simpy.Container(simulation, init=financial_cap_data['fema aid'])
        
class BuiltCapital(): # --% created a separate class for just financial capitals %--
        def __init__(self, simulation, asset):
            self.setYearBuilt(asset)
            self.setValue(asset)
            self.setDamageState(asset)  
            self.setDamageValue(asset) 
            self.setInspection(simulation, asset)
        def setYearBuilt(self, asset):
            self.age = asset['Year Built']
        def setValue(self, asset):
            self.value = asset['Value']
        def setDamageState(self, asset):
            self.damage_state = asset['Damage State']
        def setDamageValue(self, asset):
            self.damage_value = asset['Damage Value']
        def setInspection(self, simulation, asset):
            self.inspection = simulation.event()
            self.inspect_start = None
            self.inspect_stop = None

class Building(BuiltCapital):
    def __init__(self, simulation, building):
        self.setAddress(building)
#         self.setResident(building)
        try:
            self.setOccupancy(building)
        except AttributeError:
            print('Invalid occupancy type provided: {0}.'.format(building['occupancy']))
        self.setMonthlyCost(building)
        self.setYearBuilt(asset)
        self.setValue(building)
        self.setDamageState(building)  
        self.setDamageValue(building) 
        self.setInspection(simulation, building)
    def setAddress(self, building):
        self.address = building['Address']
    def setOccupancy(self, building):
        self.occupancy = building['Occupancy']
    def setMonthlyCost(self, building):
        self.cost = building['Cost']
    def setBuildingArea(self, building):
        self.cost = building['Area']

        
class Residence(Building):
    #Can verify that occupancy types only relate to residences
    def __init__(self, simulation, residence):
        self.setAddress(residence)
        self.setBuildingArea(residence)
        self.setMonthlyCost(residence)
        self.setOccupancy(residence)
        self.setBedrooms(residence)
        self.setBathrooms(residence)
        self.setYearBuilt(residence)
        self.setValue(residence)
        self.setDamageState(residence)  
        self.setDamageValue(residence) 
        self.setInspection(simulation, residence)
    def setOccupancy(self, residence):
        if residence['Occupancy'] in ('Single Family Dwelling', 'Multi Family Dwelling', 'Mobile Home', 'Condo'):
            self.occupancy = residence['Occupancy']
        else:
            raise AttributeError(residence['Occupancy'])
    def setBedrooms(self, residence):
        self.bedrooms = residence['Bedrooms']
    def setBathrooms(self, residence):
        self.bathrooms = residence['Bathrooms']



## `requests.py` ##

Changed sequence of arguments in the function definitions to be simulation, capitals, entity, callbacks)

Started using self.money_to_rebuild to track money

In [20]:
# from desaster.config import inspection_time, adjuster_time, fema_process_time
# from desaster.config import engineering_assessment_time, loan_process_time
# from desaster.config import permit_process_time

def inspection(simulation, human_capital, entity, callbacks = None):
    """Request an inspection, do inspection, update entity attribute times.

    Keyword Arguments:
    entity -- An entity object from the Entity() class. Must have a value for
              attribute 'insurance_coverage', which should be set at __init__()

    simulation -- A simpy.Environment() object. This references the simulation
                  environment, and is usually set as the first variable in a
                  simulation, e.g. simulation = simpy.Environment().

    callbacks -- a generator function containing any processes you want to start
                 after the completion of the insurance claim. If this does not
                 contain a yield (therefore isn't a generator), simpy will throw
                 an error. Defaults to None.

    Returns or Attribute Changes:

    entity.inspection_put -- Record time of inspection request
    entity.inspection_get -- Record time of inspection completion
    entity.story -- append natural language summary of process
    """

    # Put in request for an inspector (shared resource)
    entity.inspection_put = simulation.now
    
    # ??? the inspection_put attribute can be added after init, ???
    request = human_capital.inspectors.request()
    yield request

    # Duration of inspection
    yield simulation.timeout(inspection_time)
    
    # The time that the inspection has been completed
    entity.inspection_get = simulation.now

    # --% Added release statement
    human_capital.inspectors.release(request)

    #write their story
    entity.story.append(
    "{1}'s house was inspected {0} days after the event. ".format(entity.inspection_get, entity.name))
    
    entity.story.append(
                    'The event caused ${0} of damage to the residence. '.format(entity.residence.damage_value)
                    )

    if callbacks is not None:
        yield simulation.process(callbacks)
    else:
        pass

def insurance_claim(simulation, human_capital, entity, callbacks = None):
    """File an insurance claim, assign claim amounts to entity objects.

    Keyword arguments:
    entity -- An entity object from the Entity() class. Must have a value for
              attribute 'insurance_coverage', which should be set at __init__()

    simulation -- A simpy.Environment() object. This references the simulation
                  environment, and is usually set as the first variable in a
                  simulation, e.g. simulation = simpy.Environment().

    callbacks -- a generator function containing any processes you want to start
                 after the completion of the insurance claim. If this does not
                 contain a yield (therefore isn't a generator), simpy will throw
                 an error. Defaults to None.

    Returns or attribute changes:

    entity.claim_put -- Record current simulation time at the time the entity
                        enters the adjuster queue

    entity.claim_payout -- Set claim payout equal to damage value amount.

    entity.claim_get -- Record simulation time when entity recieves payout

    entity.story -- Append natural language sentences to entities story.
    """
    try: # in case a process interrupt is thrown in a master process
        
        if entity.money_to_rebuild >= entity.residence.damage_value:  # Doesn't need to make a claim
            
            return
        
        elif entity.insurance == 0.0:  # Has no insurance
            
            return
        
        else: # Has insurance and needs to submit a claim
            
            entity.claim_put = simulation.now   # Record time that claim is put in
            
            entity.story.append(
                '{0} submitted an insurance claim {1} days after the event. '.format(
                entity.name, entity.claim_put))
            
            request = human_capital.insurance_adjusters.request()
            yield request

            yield simulation.timeout(adjuster_time)    # Duration of claim processing

            
            entity.claim_get = simulation.now     # Record when the time when household gets claim payout

            human_capital.insurance_adjusters.release(request)
            
            # Only payout amount equal to the damage, not the full coverage
            if entity.residence.damage_value < entity.insurance:
                entity.claim_payout = entity.residence.damage_value
            else:
                entity.claim_payout = entity.insurance

            entity.money_to_rebuild += entity.claim_payout

            entity.story.append(
                '{0} received a ${1} insurance payout {2} days after the event. '.format(
                entity.name, entity.claim_payout, entity.claim_get))
       
    except simpy.Interrupt as i: # Handle any interrupt thrown by a master process
        
        entity.story.append(
                '{0} gave up during the insurance claim process after a {1} day search for money. '.format(
                entity.name, i.cause))
    
    if callbacks is not None:
        yield simulation.process(callbacks)

    else:
        pass

def fema_assistance(simulation, human_capital, financial_capital, entity, callbacks = None):
    """Request and receive assistance from fema.

    entity -- An entity object from the Entity() class. Must have a value for
              attributes: 'residence.damage_value', 'claim_payout',
              which should be set at __init__()

    simulation -- A simpy.Environment() object. This references the simulation
                  environment, and is usually set as the first variable in a
                  simulation, e.g. simulation = simpy.Environment().

    callbacks -- a generator function containing any processes you want to start
                 after the completion of the insurance claim. If this does not
                 contain a yield (therefore isn't a generator), simpy will throw
                 an error. Defaults to None.


    Returns or Attribute Changes:

    entity.assistance_put -- Records sim time of fema processor request

    entity.assistance_get -- Records sim time of fema assistance reciept

    entity.assistance_request -- The amount of money being requested by the
                                 entity, equal to the residence.damage_value minus the
                                 claim_payout provided by the insurance process

    entity.assistance_payout -- amount of money given to the entity, equal to
                                the request amount or whatever is left in the
                                financial_capital.fema_aid container, whichever is higher.

    """

    try:

        if entity.money_to_rebuild >= entity.residence.damage_value: # Doesn't need FEMA assistance
            return

        else: # Requires FEMA assistance
            
            entity.assistance_put = simulation.now  # Record time household requests FEMA assistance

            entity.story.append(
                '{0} submitted a request to FEMA {1} days after the event. '.format(
                entity.name, entity.assistance_put))

            request = human_capital.fema_processors.request()
            yield request
            
            yield simulation.timeout(fema_process_time)  # Duration required for FEMA to process assistance request

            human_capital.fema_processors.release(request)

            entity.assistance_get = simulation.now   # Record time household gets FEMA assistance

            # Compute amount of assistance requested from FEMA; if insurance payout covers repair cost it is zero
            entity.assistance_request = entity.residence.damage_value - entity.claim_payout

            # If requesting assistance, determine if FEMA has money left to provide assistance
            if entity.assistance_request <= financial_capital.fema_aid.level: # FEMA has enough money
                entity.assistance_payout = entity.assistance_request
                entity.money_to_rebuild += entity.assistance_payout

                yield financial_capital.fema_aid.get(entity.assistance_request)
                
                # Write the household's story
                entity.story.append(
                    '{0} received ${1} from FEMA {2} days after the event. '.format(
                    entity.name, entity.assistance_payout, entity.assistance_get))

            elif financial_capital.fema_aid.level > 0:  # FEMA has money left but less than requested
                entity.assistance_payout = financial_capital.fema_aid.level
                entity.money_to_rebuild += entity.assistance_payout
                
                yield financial_capital.fema_aid.get(financial_capital.fema_aid.level)
                
                # Write the household's story
                entity.story.append(
                 '{0} requested ${1} from FEMA but only received ${2}. '
                 .format(entity.name, entity.assistance_request, entity.assistance_payout))

                entity.story.append(
                    'They received the assistance {0} days after the event. '.format(
                    entity.assistance_get))
    
            else:  # FEMA has no money left
                
                entity.assistance_payout = 0.0

                # Write the household's story
                entity.story.append(
                '{0} received no money from FEMA because of inadequate funding. '
                .format(entity.name))
                
                yield financial_capital.fema_aid.get(financial_capital.fema_aid.level)
            
    # Catch any interrupt from a master process         
    except simpy.Interrupt as i:
        
        entity.story.append(
                '{0} gave up during the FEMA assistance process after a {1} day search for money. '.format(
                entity.name, i.cause))
        
    if callbacks is not None:
        yield simulation.process(callbacks)

    else:
        pass

def engineering_assessment(simulation, human_capital, entity, callbacks = None):
    """Request an engineering assessment"""

    entity.assessment_put = simulation.now #time of request
    
    request = human_capital.engineers.request()
    yield request

    yield simulation.timeout(engineering_assessment_time)
    
    human_capital.engineers.release(request)
    
    entity.assessment_get = simulation.now #when assessment is received

    entity.story.append(
    '{0} received an engineering assessment {1} days after the event. '
    .format(entity.name, entity.assessment_get))

    if callbacks is not None:
        yield simulation.process(callbacks)
    else:
        pass

def loan(simulation, human_capital, entity, callbacks = None):
    
    try:

        if entity.money_to_rebuild >= entity.residence.damage_value:  # Don't need a loan
            return
        
        else:
            entity.loan_put = simulation.now # Record time application submitted
                 
            entity.story.append(
                '{0} submitted a loan application {1} days after the event. '.format(
                entity.name, entity.loan_put))

            request = human_capital.loan_processors.request()
            yield request

            yield simulation.timeout(loan_process_time)  # Duration of loan processing

            human_capital.loan_processors.release(request)

            entity.loan_get = simulation.now


            # --% Added initial code to produce loan payout %--
            # TODO Need code here to determine how much money the entity requests for their
            #loan and whether the are approved
            # TODO make sure to add to entity.money_to_rebuild

            entity.loan_amount = entity.residence.damage_value - entity.claim_payout - entity.assistance_payout

            if entity.loan_amount > 0.0:
                entity.money_to_rebuild += entity.loan_amount

                entity.story.append(
                "{0} received a loan for ${1} {2} days after the event. "
                .format(entity.name, entity.loan_amount, entity.loan_get))

    # Handle any interrupt from a master process
    except simpy.Interrupt as i:
        
        entity.story.append(
                '{0} gave up during the loan approval process after a {1} day search for money. '.format(
                entity.name, i.cause))
    
    if callbacks is not None:
        yield simulation.process(callbacks)
    else:
        pass

def permit(simulation, human_capital, entity, callbacks = None):

    """Request a permit for building."""
    entity.permit_put = simulation.now

    request = human_capital.permit_processors.request()
    yield request

    yield simulation.timeout(permit_process_time)

    # --% Added release statement
    human_capital.permit_processors.release(request)

    entity.permit_get = simulation.now

    entity.story.append(
    "{0} received permit approval {1} days after the event. "
    .format(entity.name, entity.permit_get))

    if callbacks is not None:
        yield simulation.process(callbacks)
    else:
        pass

## `rebuild.py` ##

Made argument refer to human_capital, rather than directly to contractors, to be consistent with other functions.

In [21]:
# from desaster.config import sfr_rebuild_time

def rebuild_house(simulation, human_capital, entity): 
        
        if entity.permit_get > 0.0 and entity.money_to_rebuild >= entity.residence.damage_value:
            # Put in request for contractors to repair house
            entity.house_put = simulation.now

            request = human_capital.contractors.request()
            yield request
            
            # Time required to rebuild house
            if entity.residence.occupancy == 'Single Family Dwelling':
                yield simulation.timeout(sfr_rebuild_time)
            if entity.residence.occupancy == 'Multi Family Dwelling':
                yield simulation.timeout(mfr_rebuild_time)
            if entity.residence.occupancy == 'Mobile Home':
                yield simulation.timeout(mobile_rebuild_time)

            human_capital.contractors.release(request)


            # Record time when household gets house
            entity.house_get = simulation.now

            # Write the household's story
            entity.story.append(
                'The house was rebuilt {0} days after the event, taking {1} days to rebuild. '.format(
                entity.house_get, sfr_rebuild_time))
        elif entity.money_to_rebuild < entity.residence.damage_value:
            entity.story.append(
                '{0} was unable to get enough money to rebuild. '.format(
                entity.name))
        else: 
            entity.story.append(
                '{0} was unable to get a permit to rebuild. '.format(
                entity.name))

## `search.py` ##

## `config.py` ##

In [22]:
#import random

random.seed(69)

inspection_mean = 10.0
inspection_std = 0
inspection_time = abs(random.gauss(inspection_mean, inspection_std))

adjuster_mean = 15.0
adjuster_std = 0.0
adjuster_time = abs(random.gauss(adjuster_mean, adjuster_std))

fema_process_mean = 20.0
fema_process_std = 0.0
fema_process_time = abs(random.gauss(fema_process_mean, fema_process_std))

engineering_mean = 25.0
engineering_std = 0.0
engineering_assessment_time = abs(random.gauss(engineering_mean, engineering_std))

loan_process_mean = 30.0
loan_process_std = 0.0
loan_process_time = abs(random.gauss(loan_process_mean, loan_process_std))

permit_process_mean = 35.0
permit_process_std = 0.0
permit_process_time = abs(random.gauss(permit_process_mean, permit_process_std))

mobile_rebuild_mean = 40.0
mobile_rebuild_std = 0.0
mobile_rebuild_time = abs(random.gauss(mobile_rebuild_mean, mobile_rebuild_std))

sfr_rebuild_mean = 45.0
sfr_rebuild_std = 0.0
sfr_rebuild_time = abs(random.gauss(sfr_rebuild_mean, sfr_rebuild_std))

mfr_rebuild_mean = 50.0
mfr_rebuild_std = 0.0
mfr_rebuild_time = abs(random.gauss(mfr_rebuild_mean, mfr_rebuild_std))

## `io.py` ##

In [23]:
# from desaster import config


def view_config():
    var = vars(config)
    conf = dir(config)
    result = {}
    for i in conf:
        if "__" not in i and "random" not in i:
            print ("{} = {}".format(i, var[i]))
            #print (i)
            result[i] = var[i]
            
        else:
            pass
    return result

def imp_concat():
    input1 = pd.read_csv("../inputs/Work out of County Households.csv")
    input2 = pd.read_csv("../inputs/Work in County Households.csv")
    input3 = pd.read_csv("../inputs/Middle High School Households.csv")
    input4 = pd.read_csv("../inputs/Elementary School Households.csv")
    input5 = pd.read_csv("../inputs/Grocery Households.csv")
    all_inputs = [input1, input2, input3, input4, input5]
    data = pd.concat(all_total)
    return data

def get_names():
    lasts = pd.read_csv("../inputs/last_names.csv")
    data = lasts.iloc[0:1366]
    return data['name']

## Simulation Testing ##

In [24]:
simulation = simpy.Environment()

In [25]:
def find_rebuild_money(simulation, human_capital, financial_capital, entity):
    try:
        
        patience_length = 290.0
        start_money_search = simulation.now
        patience_end = simulation.now + patience_length
        
        if entity.insurance > 0.0:
        
            find_money_patience = simulation.timeout(patience_end - simulation.now, value='Gave up')
            try_insurance = simulation.process(insurance_claim(simulation, human_capital, entity))
            
            money_search_outcome = yield find_money_patience | try_insurance
            
            if money_search_outcome == {find_money_patience: 'Gave up'}:
                
                try_insurance.interrupt(simulation.now - start_money_search)


                return
        
        if entity.money_to_rebuild < entity.residence.damage_value:
            
            find_money_patience = simulation.timeout(patience_end - simulation.now, value='Gave up')
            try_fema = simulation.process(fema_assistance(simulation, human_capital, financial_capital, entity))
            
            money_search_outcome = yield find_money_patience | try_fema
            
            if money_search_outcome == {find_money_patience: 'Gave up'}:
                try_fema.interrupt(simulation.now - start_money_search)

                return
            
        if entity.money_to_rebuild < entity.residence.damage_value:
            
            find_money_patience = simulation.timeout(patience_end - simulation.now, value='Gave up')
            try_loan = simulation.process(loan(simulation, human_capital, entity))
            
            money_search_outcome = yield find_money_patience | try_loan
            
            if money_search_outcome == {find_money_patience: 'Gave up'}:
                try_loan.interrupt(simulation.now - start_money_search)

                return

        entity.story.append(
                                'It took {0} {1} days to exhaust the search for money. '.format(
                                entity.name, simulation.now - start_money_search))

        return
    
    except:
        
        entity.story.append(
                            '{0} gave up searching for money after {1} days. '.format(
                                entity.name, simulation.now - start_money_search))
        
        return

In [26]:
def master_process(simulation, human_capital, financial_capital, entity):
       
    yield simulation.process(inspection(simulation, human_capital, entity))
    
    if entity.money_to_rebuild < entity.residence.damage_value:
        yield simulation.process(find_rebuild_money(simulation, human_capital, financial_capital, entity))

    if entity.money_to_rebuild >= entity.residence.damage_value:
        yield simulation.process(engineering_assessment(simulation, human_capital, entity))

        yield simulation.process(permit(simulation, human_capital, entity))

        yield simulation.process(rebuild_house(simulation, human_capital, entity))

    return
   

Now must separate define human_cap_data and financial_cap_data ....previously just "data"

In [27]:
human_cap_data = {
        "inspectors": 4,
        "insurance adjusters": 4,
        "fema processors": 4,
        "permit processors": 4,
        "contractors": 4,
        "loan processors": 4,
        "engineers": 4
                 }

In [28]:
financial_cap_data = {"fema aid": 350000000}
# financial_cap_data = {"fema aid": 0}

No longer make call to Capitals() becuase make separate classes for each type %--

Formerly: `res = desaster.Capitals(sim, human_cap_data) # Need reference to desaster. for packages`

In [29]:
human_capital = HumanCapital(simulation, human_cap_data) 

Now also need to make call to FinancialCapital (e.g., for fema aid)

In [30]:
financial_capital = FinancialCapital(simulation, financial_cap_data) 

Read in household data. This data file contains attributes for the household *AND* the household's residence. At the moment this prevents a household from owning multiple residences, which will require using separate tables and doing a lookup to assign properties.

In [31]:
file_path = "../inputs/household_inputs_test.csv"
households_df = pd.read_csv(file_path)
households_df.head()

,Name,Income,Savings,Insurance,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State,Damage Value
0,Alfred,30000,10000,0,62 That St,Mobile Home,100,1,1,700,1920,100000,None,20000
1,Bruce,100000,1000000,10000000,720 This Rd,Single Family Dwelling,100000,6,5,5000,1920,10000000,Slight,50000
2,Selena,10000,100,0,1001 Other Ave,Multi Family Dwelling,10,0,1,250,1960,10000,Complete,8000
3,Fish,50000,1000,550000,26000 Out There Lane,Single Family Dwelling,2000,4,2,2000,2010,800000,Moderate,650000


Instantiate household objects from Household()

When packaging must reference Household() as entities.Household()

In [32]:
household = {}
for i in households_df.index:
    household[i] = Household(simulation, households_df.iloc[i])

In [33]:
for i in range(len(household)):
    #print (household[i])
    simulation.process(master_process(simulation, human_capital, financial_capital, household[i]))

In [34]:
simulation.run()

In [35]:
household[0].story

['Alfred lives in a 1 bedroom Mobile Home Home (62 That St). ',
 "Alfred's house was inspected 10.0 days after the event. ",
 'The event caused $20000 of damage to the residence. ',
 'Alfred submitted a request to FEMA 10.0 days after the event. ',
 'Alfred received $20000.0 from FEMA 30.0 days after the event. ',
 'It took Alfred 20.0 days to exhaust the search for money. ',
 'Alfred received an engineering assessment 55.0 days after the event. ',
 'Alfred received permit approval 90.0 days after the event. ',
 'The house was rebuilt 130.0 days after the event, taking 45.0 days to rebuild. ']

In [36]:
household[1].story

['Bruce lives in a 6 bedroom Single Family Dwelling Home (720 This Rd). ',
 "Bruce's house was inspected 10.0 days after the event. ",
 'The event caused $50000 of damage to the residence. ',
 'Bruce received an engineering assessment 35.0 days after the event. ',
 'Bruce received permit approval 70.0 days after the event. ',
 'The house was rebuilt 115.0 days after the event, taking 45.0 days to rebuild. ']

In [37]:
household[2].story

['Selena lives in a 0 bedroom Multi Family Dwelling Home (1001 Other Ave). ',
 "Selena's house was inspected 10.0 days after the event. ",
 'The event caused $8000 of damage to the residence. ',
 'Selena submitted a request to FEMA 10.0 days after the event. ',
 'Selena received $8000.0 from FEMA 30.0 days after the event. ',
 'It took Selena 20.0 days to exhaust the search for money. ',
 'Selena received an engineering assessment 55.0 days after the event. ',
 'Selena received permit approval 90.0 days after the event. ',
 'The house was rebuilt 140.0 days after the event, taking 45.0 days to rebuild. ']

In [38]:
household[3].story

['Fish lives in a 4 bedroom Single Family Dwelling Home (26000 Out There Lane). ',
 "Fish's house was inspected 10.0 days after the event. ",
 'The event caused $650000 of damage to the residence. ',
 'Fish submitted an insurance claim 10.0 days after the event. ',
 'Fish received a $550000 insurance payout 25.0 days after the event. ',
 'Fish submitted a request to FEMA 25.0 days after the event. ',
 'Fish received $100000 from FEMA 45.0 days after the event. ',
 'It took Fish 35.0 days to exhaust the search for money. ',
 'Fish received an engineering assessment 70.0 days after the event. ',
 'Fish received permit approval 105.0 days after the event. ',
 'The house was rebuilt 150.0 days after the event, taking 45.0 days to rebuild. ']

In [63]:
file_path = "../inputs/bldg_struct_repair_cost_ratios.csv"
bldg_struct_repair_costs = pd.read_csv(file_path, index_col='Occupancy Class')


In [64]:
struct_repair_costs

,No.,Label,Slight,Moderate,Extensive,Complete
Occupancy Class,,,,,,
Single Family Dwelling,1,RES1,0.5,2.3,11.7,23.4
Mobile Home,2,RES2,0.4,2.4,7.3,24.4
Multi Family Dwelling,3-8,RES3a-\rf,0.3,1.4,6.9,13.8
Temporary Lodging,9,RES4,0.2,1.4,6.8,13.6
Institutional Dormitory,10,RES5,0.4,1.9,9.4,18.8
Nursing Home,11,RES6,0.4,1.8,9.2,18.4
Retail Trade,12,COM1,0.6,2.9,14.7,29.4
Wholesale Trade,13,COM2,0.6,3.2,16.2,32.4
Personal and Repair Services,14,COM3,0.3,1.6,8.1,16.2


In [48]:
household[3].residence.occupancy

'Single Family Dwelling'

In [49]:
i = 1
print(household[i].residence.occupancy)
print(household[i].residence.damage_state)
bldg_struct_damage_ratio = bldg_struct_repair_costs.ix[household[i].residence.occupancy][household[i].residence.damage_state] / 100.0
print(struct_damage_ratio)

Single Family Dwelling
Slight
0.005


In [50]:
household[3].residence.damage_value = struct_damage_ratio * household[3].residence.value

In [51]:
household[3].residence.damage_value

4000.0

In [59]:
def setDamageValue(building):
    if building.damage_state != 'None':
        repair_ratios_file = "../inputs/bldg_struct_repair_cost_ratios.csv"
        repair_ratios_table = pd.read_csv(repair_ratios_file)
        repair_ratio = repair_ratios_table.ix[building.occupancy][building.damage_state] / 100.0
        residence.damage_value = repair_ratio * building.value

In [62]:
repair_ratios_file = "../inputs/bldg_struct_repair_cost_ratios.csv"
repair_ratios_table = pd.read_csv(repair_ratios_file)
repair_ratios_table

,No.,Label,Occupancy Class,Slight,Moderate,Extensive,Complete
0,1,RES1,Single Family Dwelling,0.5,2.3,11.7,23.4
1,2,RES2,Mobile Home,0.4,2.4,7.3,24.4
2,3-8,RES3a-\rf,Multi Family Dwelling,0.3,1.4,6.9,13.8
3,9,RES4,Temporary Lodging,0.2,1.4,6.8,13.6
4,10,RES5,Institutional Dormitory,0.4,1.9,9.4,18.8
5,11,RES6,Nursing Home,0.4,1.8,9.2,18.4
6,12,COM1,Retail Trade,0.6,2.9,14.7,29.4
7,13,COM2,Wholesale Trade,0.6,3.2,16.2,32.4
8,14,COM3,Personal and Repair Services,0.3,1.6,8.1,16.2
9,15,COM4,Professional/Technical/ \rBusiness Services,0.4,1.9,9.6,19.2


In [60]:
setDamageValue(household[3].residence)

KeyError: 'Single Family Dwelling'